In [1]:
with open("data/input.txt", "r", encoding="utf-8") as f:
    text = f.read()

In [2]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115389


In [3]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [4]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print("".join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [5]:
stoi = { ch:i for i, ch in enumerate(chars) }
itos = { i:ch for i, ch in enumerate(chars) }

# Convert a string to a list of integers
encode = lambda s: [stoi[c] for c in s]

# Given a list of integers convert to a string
decode = lambda l: "".join([itos[i] for i in l])

In [6]:
encoded_message = encode("some message to encode")
print(encoded_message)

decoded_messaged = decode(encoded_message)
print(decoded_messaged)

[57, 53, 51, 43, 1, 51, 43, 57, 57, 39, 45, 43, 1, 58, 53, 1, 43, 52, 41, 53, 42, 43]
some message to encode


In [14]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1337)

In [8]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115389]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [9]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [10]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [11]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context}, the target is: {target}")

when input is tensor([18]), the target is: 47
when input is tensor([18, 47]), the target is: 56
when input is tensor([18, 47, 56]), the target is: 57
when input is tensor([18, 47, 56, 57]), the target is: 58
when input is tensor([18, 47, 56, 57, 58]), the target is: 1
when input is tensor([18, 47, 56, 57, 58,  1]), the target is: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]), the target is: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]), the target is: 58


In [12]:
# how many independent sequences will we process in parallel?
batch_size = 4
# what is the maximum context length for predictions?
block_size = 8

def get_batch(split: str):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[59, 57,  1, 58, 56, 39, 47, 58],
        [30, 10,  0, 15, 53, 51, 43,  6],
        [34, 21, 26, 15, 17, 26, 32, 21],
        [27, 10,  0, 32, 46, 53, 59,  1]])
targets:
torch.Size([4, 8])
tensor([[57,  1, 58, 56, 39, 47, 58, 53],
        [10,  0, 15, 53, 51, 43,  6,  1],
        [21, 26, 15, 17, 26, 32, 21, 27],
        [10,  0, 32, 46, 53, 59,  1, 42]])
----
when input is [59] the target: 57
when input is [59, 57] the target: 1
when input is [59, 57, 1] the target: 58
when input is [59, 57, 1, 58] the target: 56
when input is [59, 57, 1, 58, 56] the target: 39
when input is [59, 57, 1, 58, 56, 39] the target: 47
when input is [59, 57, 1, 58, 56, 39, 47] the target: 58
when input is [59, 57, 1, 58, 56, 39, 47, 58] the target: 53
when input is [30] the target: 10
when input is [30, 10] the target: 0
when input is [30, 10, 0] the target: 15
when input is [30, 10, 0, 15] the target: 53
when input is [30, 10, 0, 15, 53] the target: 51
when input is [30, 

In [23]:
class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (batch, time, channels) / channels is vocab_size

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

idx = torch.zeros((1, 1), dtype=torch.long)
tokens = m.generate(idx, max_new_tokens=100)[0].tolist()
print(decode(tokens))

torch.Size([32, 65])
tensor(4.4109, grad_fn=<NllLossBackward0>)

Tm'd'!-SH'ojQWIcjo-hpWfeKT$Re'eUqj.RTyuAZyCfjTw;CS,'RfpvsKJoSQmLmXB:l
erh?f. dZOd'vv'sOixZia$vVStWy-


In [24]:
# Generally 1e-4 is used but our NN is simple so we can use 1e-3
# https://pytorch.org/docs/stable/generated/torch.optim.AdamW.html
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

/Users/calvin/.pyenv/versions/3.12.1/envs/ml/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [37]:
batch_size = 32
for steps in range(10000):
    # sample a batch of data
    xb, yb = get_batch("train")

    # evaluate the loss
    logits, loss = m(xb, yb)
    # https://pytorch.org/docs/stable/generated/torch.optim.Optimizer.zero_grad.html
    optimizer.zero_grad(set_to_none=True)
    # https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html
    loss.backward()
    # https://pytorch.org/docs/stable/generated/torch.optim.Optimizer.step.html
    optimizer.step()


print(loss.item())

2.4470670223236084


In [41]:
print(decode(m.generate(torch.zeros((1, 1), dtype=torch.long), max_new_tokens=300)[0].tolist()))



Sw fru t f msot coupondariores KES:

Kise'druspino w

S:
AULA:
ILORY INThe, o hal birve,
Gu m ngr s trig,
LOf tin nte wit ethanee cas te latl tofr icond tol, Byoue!
Tordeount bero fur;
Ho t aifean.


IFo myoulfokedsh nd bllellos I win matthiere tisoongise, it who make nougofaus, d y:
WAnd meeilande
